In [ ]:
#hide   
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
from datetime import datetime as dt
from typing import *
from rfpye.constants import *
from rfpye.parser import *
from rfpye.utils import *
from rfpye.blocks import *
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.basics import partialler
from fastcore.parallel import parallel
import pandas as pd
import numpy as np
from geopy.distance import geodesic

%load_ext autoreload
%autoreload 2 

In [ ]:
entrada = Path(r'D:\OneDrive - ANATEL\Remote')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

In [ ]:
arquivos = get_files(entrada, extensions=['.bin']) ;  arquivos

(#65) [Path('D:/OneDrive - ANATEL/Remote/rfeye002073/rfeye002073_210620_T231206.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002080/rfeye002080_691231_T210111.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002081/rfeye002081_210620_T232204.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002083/rfeye002083_210621_T160001.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002084/rfeye002084_210623_T144012.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002086/rfeye002086_210622_T004723.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002087/rfeye002087_210628_T224301.bin'),Path('D:/OneDrive - ANATEL/Remote/RFeye002090-VCP/rfeye002090-VCP_210623_T024236.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002091/rfeye002091_210618_T145747.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002092/rfeye002092_210603_T205009.bin')...]

In [ ]:
%%time
btypes = [40]
func = partialler(parse_bin, btypes=btypes)
parsed_bins = parallel(func, arquivos[:5], n_workers=8, pause=0.25)

Wall time: 3.59 s


In [ ]:
metas = parallel(export_metadata, parsed_bins, n_workers=8, pause=0.25)

In [ ]:
metas

(#5) [{40:                                altitude  data_size        gps_datetime  \
wallclock_datetime                                                       
2021-06-20 23:13:00.192500  1098.500000         40 2021-06-20 23:12:59   
2021-06-20 23:14:00.546600  1097.099976         40 2021-06-20 23:13:59   
2021-06-20 23:15:00.901100  1098.800049         40 2021-06-20 23:14:59   
2021-06-20 23:16:00.885900  1098.300049         40 2021-06-20 23:15:59   
2021-06-20 23:17:00.695800  1098.400024         40 2021-06-20 23:16:59   
...                                 ...        ...                 ...   
2021-06-27 06:05:00.185800  1096.099976         40 2021-06-27 06:04:59   
2021-06-27 06:06:00.194300  1096.199951         40 2021-06-27 06:05:59   
2021-06-27 06:07:00.178000  1100.599976         40 2021-06-27 06:06:59   
2021-06-27 06:08:00.178600  1095.300049         40 2021-06-27 06:07:59   
2021-06-27 06:09:00.177800  1094.199951         40 2021-06-27 06:08:59   

                          

In [ ]:
gps = {filename.parent.name: meta[40] for filename, meta in zip(arquivos, metas) if 40 in meta}

In [ ]:
def agg_coords(df):
    coord = df[['latitude', 'longitude']].aggregate(['mean', 'median', 'std'])
    coord.loc['weighted_mean', 'latitude'] = (df.latitude * df.num_satellites).sum() /  df.num_satellites.sum()
    coord.loc['weighted_mean', 'longitude'] = (df.longitude * df.num_satellites).sum() /  df.num_satellites.sum()
    result = pd.DataFrame()
    cols = ['Lat_' + c for c in coord.index.tolist()] + [['Long_' + c for c in coord.index.tolist()]]
    for c in coord.columns.tolist():
        for i in coord.index.tolist():
            result.loc[0, c + '_' + i] = coord.loc[i,c]
    result['GPS_Blocks'] = df.shape[0]
    return result

In [ ]:
result = pd.DataFrame()
for sensor, df in gps.items():
    df = agg_coords(df)
    df['sensor'] = sensor
    result = result.append(df)

In [ ]:
result = result.reset_index(drop=True)

In [ ]:
result.latitude_std.max()

0.0002987931075040251

In [ ]:
estações = pd.read_excel(r'D:\OneDrive - ANATEL\GR01FI3\estações_rfeye.xlsx', sheet_name='Estações', engine='openpyxl')

In [ ]:
estações.columns = estações.columns.str.replace(' ', "_")

In [ ]:
estações = estações[estações.Tipo_de_Estação == 'RFeye']
estações = estações[estações.Situação_do_Equipamento == 'Ativo']

In [ ]:
estações = estações.loc[estações.ID_de_rede.isin(result.sensor.tolist()), ['ID_de_rede', 'Latitude', 'Longitude']].reset_index(drop=True)

In [ ]:
estações

,ID_de_rede,Latitude,Longitude
0,rfeye002073,-15.764454,-47.890370
1,rfeye002080,-14.870252,-40.863344
2,rfeye002081,-15.868921,-47.925766
3,rfeye002083,-5.764321,-35.368176
4,rfeye002084,-15.560278,-56.064167
...,...,...,...
59,rfeye002315,-3.250833,-52.252222
60,rfeye002321,-26.880687,-48.649411
61,rfeye002325,-17.796639,-50.929681
62,rfeye002332,-22.767000,-45.522664


In [ ]:
for row in result.itertuples():
    result.loc[row.Index, 'latitude_sharepoint'] = estações.loc[estações.ID_de_rede == row.sensor, 'Latitude'].item()
    result.loc[row.Index, 'longitude_sharepoint'] = estações.loc[estações.ID_de_rede == row.sensor, 'Longitude'].item()

In [ ]:
result

,latitude_mean,latitude_median,latitude_std,latitude_weighted_mean,longitude_mean,longitude_median,longitude_std,longitude_weighted_mean,GPS_Blocks,sensor,latitude_sharepoint,longitude_sharepoint
0,-15.764438,-15.764439,0.000013,-15.764440,-47.890373,-47.890373,0.000015,-47.890379,9057,rfeye002073,-15.764454,-47.890370
1,-14.870231,-14.870230,0.000018,-14.870231,-40.863346,-40.863342,0.000021,-40.863343,9061,rfeye002080,-14.870252,-40.863344
2,-15.868937,-15.868938,0.000014,-15.868936,-47.925781,-47.925781,0.000017,-47.925783,9057,rfeye002081,-15.868921,-47.925766
3,-5.764397,-5.764399,0.000022,-5.764396,-35.368584,-35.368584,0.000019,-35.368582,9060,rfeye002083,-5.764321,-35.368176
4,-15.561007,-15.561005,0.000011,-15.561005,-56.063992,-56.063992,0.000012,-56.063990,9027,rfeye002084,-15.560278,-56.064167
...,...,...,...,...,...,...,...,...,...,...,...,...
59,-3.252711,-3.252711,0.000009,-3.252711,-52.248211,-52.248207,0.000009,-52.248213,9057,rfeye002315,-3.250833,-52.252222
60,-26.880144,-26.880144,0.000014,-26.880143,-48.648079,-48.648079,0.000014,-48.648080,9059,rfeye002321,-26.880687,-48.649411
61,-17.796627,-17.796629,0.000011,-17.796628,-50.929688,-50.929684,0.000012,-50.929679,9056,rfeye002325,-17.796639,-50.929681
62,-22.767006,-22.767006,0.000018,-22.767006,-45.522652,-45.522652,0.000018,-45.522646,9058,rfeye002332,-22.767000,-45.522664


In [ ]:
result = result.drop(['latitude_std', 'longitude_std'], axis=1)

In [ ]:
result = result.set_index('sensor') ; result

,latitude_mean,latitude_median,latitude_weighted_mean,longitude_mean,longitude_median,longitude_weighted_mean,GPS_Blocks,latitude_sharepoint,longitude_sharepoint
sensor,,,,,,,,,
rfeye002073,-15.764438,-15.764439,-15.764440,-47.890373,-47.890373,-47.890379,9057,-15.764454,-47.890370
rfeye002080,-14.870231,-14.870230,-14.870231,-40.863346,-40.863342,-40.863343,9061,-14.870252,-40.863344
rfeye002081,-15.868937,-15.868938,-15.868936,-47.925781,-47.925781,-47.925783,9057,-15.868921,-47.925766
rfeye002083,-5.764397,-5.764399,-5.764396,-35.368584,-35.368584,-35.368582,9060,-5.764321,-35.368176
rfeye002084,-15.561007,-15.561005,-15.561005,-56.063992,-56.063992,-56.063990,9027,-15.560278,-56.064167
...,...,...,...,...,...,...,...,...,...
rfeye002315,-3.252711,-3.252711,-3.252711,-52.248211,-52.248207,-52.248213,9057,-3.250833,-52.252222
rfeye002321,-26.880144,-26.880144,-26.880143,-48.648079,-48.648079,-48.648080,9059,-26.880687,-48.649411
rfeye002325,-17.796627,-17.796629,-17.796628,-50.929688,-50.929684,-50.929679,9056,-17.796639,-50.929681


In [ ]:
for row in result.itertuples():
    result.loc[row.Index, 'Distância'] = np.round(geodesic((row.latitude_sharepoint , row.longitude_sharepoint), (row.latitude_weighted_mean, row.longitude_weighted_mean)).meters, 2)

In [ ]:
result = result.sort_values('Distância', ascending=False)

In [ ]:
result.head(10)

,latitude_mean,latitude_median,latitude_weighted_mean,longitude_mean,longitude_median,longitude_weighted_mean,GPS_Blocks,latitude_sharepoint,longitude_sharepoint,Distância
sensor,,,,,,,,,,
rfeye002185,-16.824139,-16.824139,-16.824140,-49.246204,-49.246201,-49.246197,9057,-16.752776,-49.210742,8755.46
rfeye002147,-9.367547,-9.367547,-9.367546,-40.563629,-40.563625,-40.563632,9050,-9.367668,-40.571085,818.83
rfeye002315,-3.252711,-3.252711,-3.252711,-52.248211,-52.248207,-52.248213,9057,-3.250833,-52.252222,491.56
rfeye002234,-7.145658,-7.145658,-7.145659,-34.948120,-34.948120,-34.948120,9057,-7.144481,-34.950880,331.58
rfeye002311,-10.296946,-10.296946,-10.296946,-48.358917,-48.358913,-48.358912,9056,-10.294337,-48.359383,293.17
rfeye002177,-3.784329,-3.784329,-3.784330,-38.529892,-38.529892,-38.529891,9057,-3.786208,-38.528264,275.34
rfeye002086,-16.016932,-16.016932,-16.016931,-48.058235,-48.058239,-48.058245,9058,-16.017009,-48.060472,238.51
rfeye002275,-2.584169,-2.584169,-2.584169,-44.237549,-44.237553,-44.237551,9058,-2.583415,-44.236225,169.43
rfeye002321,-26.880144,-26.880144,-26.880143,-48.648079,-48.648079,-48.648080,9059,-26.880687,-48.649411,145.39


In [ ]:
result.tail(10)

,latitude_mean,latitude_median,latitude_weighted_mean,longitude_mean,longitude_median,longitude_weighted_mean,GPS_Blocks,latitude_sharepoint,longitude_sharepoint,Distância
sensor,,,,,,,,,,
rfeye002143,-25.517706,-25.517702,-25.517705,-54.558212,-54.558216,-54.558209,9055,-25.517694,-54.558204,1.31
rfeye002325,-17.796627,-17.796629,-17.796628,-50.929688,-50.929684,-50.929679,9056,-17.796639,-50.929681,1.26
rfeye002180,-22.807728,-22.807726,-22.807727,-43.238640,-43.238640,-43.238642,9055,-22.807735,-43.238648,1.14
rfeye002092,-12.826848,-12.826847,-12.826850,-38.478043,-38.478043,-38.478043,9057,-12.826840,-38.478044,1.07
rfeye002283,-22.700054,-22.700052,-22.700055,-47.666859,-47.666855,-47.666858,3022,-22.700056,-47.666866,0.88
rfeye002167,-5.063365,-5.063366,-5.063364,-42.796410,-42.796413,-42.796416,9056,-5.063370,-42.796412,0.79
rfeye002221,-23.479397,-23.479397,-23.479397,-52.012199,-52.012196,-52.012198,9056,-23.479393,-52.012193,0.75
rfeye002263,-23.329048,-23.329048,-23.329047,-51.137016,-51.137016,-51.137012,9058,-23.329041,-51.137012,0.67
rfeye002179,-23.487041,-23.487043,-23.487045,-47.449852,-47.449860,-47.449857,3023,-23.487049,-47.449856,0.44


In [ ]:
result[result.Distância > 10]

(25, 10)

In [ ]:
result[result.Distância > 30]

,latitude_mean,latitude_median,latitude_weighted_mean,longitude_mean,longitude_median,longitude_weighted_mean,latitude_sharepoint,longitude_sharepoint,Distância
sensor,,,,,,,,,
rfeye002185,-16.824139,-16.824139,-16.824140,-49.246204,-49.246201,-49.246197,-16.752776,-49.210742,8755.46
rfeye002147,-9.367547,-9.367547,-9.367546,-40.563629,-40.563625,-40.563632,-9.367668,-40.571085,818.83
rfeye002315,-3.252711,-3.252711,-3.252711,-52.248211,-52.248207,-52.248213,-3.250833,-52.252222,491.56
rfeye002234,-7.145658,-7.145658,-7.145659,-34.948120,-34.948120,-34.948120,-7.144481,-34.950880,331.58
rfeye002311,-10.296946,-10.296946,-10.296946,-48.358917,-48.358913,-48.358912,-10.294337,-48.359383,293.17
rfeye002177,-3.784329,-3.784329,-3.784330,-38.529892,-38.529892,-38.529891,-3.786208,-38.528264,275.34
rfeye002086,-16.016932,-16.016932,-16.016931,-48.058235,-48.058239,-48.058245,-16.017009,-48.060472,238.51
rfeye002275,-2.584169,-2.584169,-2.584169,-44.237549,-44.237553,-44.237551,-2.583415,-44.236225,169.43
rfeye002321,-26.880144,-26.880144,-26.880143,-48.648079,-48.648079,-48.648080,-26.880687,-48.649411,145.39
